In [1]:
import uuid
import os

from db.DocumentEntry import DocumentEntry
from db.DocumentRepository import DocumentRepository
from datetime import datetime, timedelta

Let's see if the DocumentRepository works by testing the API.

Let's setup docker

In [2]:
# Connect to the database by exec docker compose in your terminal. This exec. a terminal command using Python
os.system("""
docker compose down;
docker compose up -d --build db
""")

 Container project_mse-db-1  Stopping
 Container project_mse-db-1  Stopped
 Container project_mse-db-1  Removing
 Container project_mse-db-1  Removed
 Network project_mse_default  Removing
 Network project_mse_default  Removed


#0 building with "desktop-linux" instance using docker driver

#1 [db internal] load build definition from Dockerfile
#1 transferring dockerfile: 122B done
#1 DONE 0.0s

#2 [db internal] load metadata for docker.io/library/postgres:latest
#2 DONE 0.0s

#3 [db internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [db 1/2] FROM docker.io/library/postgres:latest
#4 DONE 0.0s

#5 [db internal] load build context
#5 transferring context: 30B done
#5 DONE 0.0s

#6 [db 2/2] COPY dump.sql /docker-entrypoint-initdb.d/
#6 CACHED

#7 [db] exporting to image
#7 exporting layers done
#7 writing image sha256:3a71a465c2e6efe5bccb23028b9f1eb04b1b898db39add2d7116c1e6338e75ac done
#7 naming to docker.io/library/project_mse-db done
#7 DONE 0.0s


 Network project_mse_default  Creating
 Network project_mse_default  Created
 Container project_mse-db-1  Creating
 Container project_mse-db-1  Created
 Container project_mse-db-1  Starting
 Container project_mse-db-1  Started


0

Let's first provide the data we want to save and load

In [5]:
documentRepository = DocumentRepository()

# Get the current local date and time
now = datetime.now()

entry1 = DocumentEntry("url1", "Title 1", ["heading1"], "page text 1",
                       ["keyword1", "keyword2", "keyword3", "keyword4", "keyword5", "keyword6", "keyword7"], datetime.now(), [], ["myfirstevencoolerexternallink1", "myfirstevencoolerexternallink2"])

entry2 = DocumentEntry("url2", None, ["", None], "",
                        [None], now - timedelta(days=1), [None], [])
# entry2 = DocumentEntry("url2", [], "content2", now - timedelta(days=1))

documentsToSave = [entry1, entry2]

SC: Connected to the db. Now you can go and build the best search engine around!


Let's first clear the db

In [6]:
documentRepository.deleteAllDocuments()
allDocuments = documentRepository.loadAllDocuments()
allDocuments

SC: Deleted all documents.


[]

How does it look after we save 2 elements?

In [7]:
documentRepository.saveAllDocuments(documentsToSave)
allDocuments = documentRepository.loadAllDocuments()
print(allDocuments)
print(len(allDocuments))

SC: All documents saved.
[DocumentEntry[id=e2dfc944-b..., url=url1, title=Title 1, headings=['heading1'], page_text=page text ..., keywords=['keyword1', 'keyword2', ...], accessed_timestamp=2024-07-07 12:50:00.475927], internal_links=[]], external_links=['myfirstevencoolerexternallink1', 'myfirstevencoolerexternallink2']], DocumentEntry[id=5e00fef3-e..., url=url2, title=None, headings=['', None], page_text=, keywords=[None], accessed_timestamp=2024-07-06 12:50:00.475855], internal_links=[None]], external_links=[]]]
2


Are the types correct?

In [8]:
print(type(allDocuments))
print(type(allDocuments[0]))

<class 'list'>
<class 'db.DocumentEntry.DocumentEntry'>


The first should be "list", the second "db.DocumentEntry.DocumentEntry".

Let's add one more element

In [9]:
entry3 = DocumentEntry("url3", "Title 3", ["heading3"], "page text 3",
                       ["keyword1"], datetime.now() - timedelta(days=2), ["internalLink"], [None, None], uuid.uuid4())

# entry3 = DocumentEntry("url3", ["keyword5"], "content3", datetime.now() - timedelta(days=2), uuid.uuid4())
documentRepository.saveDocument(entry3)
allDocuments = documentRepository.loadAllDocuments()
allDocuments

SC: Saved document.


[DocumentEntry[id=e2dfc944-b..., url=url1, title=Title 1, headings=['heading1'], page_text=page text ..., keywords=['keyword1', 'keyword2', ...], accessed_timestamp=2024-07-07 12:50:00.475927], internal_links=[]], external_links=['myfirstevencoolerexternallink1', 'myfirstevencoolerexternallink2']],
 DocumentEntry[id=5e00fef3-e..., url=url2, title=None, headings=['', None], page_text=, keywords=[None], accessed_timestamp=2024-07-06 12:50:00.475855], internal_links=[None]], external_links=[]],
 DocumentEntry[id=57d20827-8..., url=url3, title=Title 3, headings=['heading3'], page_text=page text ..., keywords=['keyword1'], accessed_timestamp=2024-07-05 12:50:06.355166], internal_links=['internalLink']], external_links=[None, None]]]

Lets update this element

In [10]:
entry3 = DocumentEntry(entry3.url, entry3.title, entry3.headings, entry3.page_text,
                       ["keyword1", "keyword6"], entry3.accessed_timestamp, entry3.internal_links, entry3.external_links, entry3.id)
documentRepository.updateDocument(entry3)
allDocuments = documentRepository.loadAllDocuments()
allDocuments

SC: Updated document.


[DocumentEntry[id=e2dfc944-b..., url=url1, title=Title 1, headings=['heading1'], page_text=page text ..., keywords=['keyword1', 'keyword2', ...], accessed_timestamp=2024-07-07 12:50:00.475927], internal_links=[]], external_links=['myfirstevencoolerexternallink1', 'myfirstevencoolerexternallink2']],
 DocumentEntry[id=5e00fef3-e..., url=url2, title=None, headings=['', None], page_text=, keywords=[None], accessed_timestamp=2024-07-06 12:50:00.475855], internal_links=[None]], external_links=[]],
 DocumentEntry[id=57d20827-8..., url=url3, title=Title 3, headings=['heading3'], page_text=page text ..., keywords=['keyword1', 'keyword6'], accessed_timestamp=2024-07-05 12:50:06.355166], internal_links=['internalLink']], external_links=[None, None]]]

In [11]:
# If you want to push the current state of the database, uncomment the next line
# documentRepository.replaceDump()

Lets delete this elemente again

In [12]:
documentRepository.deleteDocument(entry3)
allDocuments = documentRepository.loadAllDocuments()
allDocuments

SC: Deleted document.


[DocumentEntry[id=e2dfc944-b..., url=url1, title=Title 1, headings=['heading1'], page_text=page text ..., keywords=['keyword1', 'keyword2', ...], accessed_timestamp=2024-07-07 12:50:00.475927], internal_links=[]], external_links=['myfirstevencoolerexternallink1', 'myfirstevencoolerexternallink2']],
 DocumentEntry[id=5e00fef3-e..., url=url2, title=None, headings=['', None], page_text=, keywords=[None], accessed_timestamp=2024-07-06 12:50:00.475855], internal_links=[None]], external_links=[]]]

Do we have 0 elements when we decide to clear the index?

In [13]:
documentRepository.deleteAllDocuments()
allDocuments = documentRepository.loadAllDocuments()
allDocuments

SC: Deleted all documents.


[]

When you get "[]", everything was right.

Finally, you should shut down the container and kill it.

In [14]:
os.system("""
docker compose down;
""")

 Container project_mse-db-1  Stopping
 Container project_mse-db-1  Stopped
 Container project_mse-db-1  Removing
 Container project_mse-db-1  Removed
 Network project_mse_default  Removing
 Network project_mse_default  Removed


0